In [201]:
pip install SQLALchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [202]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.types import Integer, Float, String, DateTime
import pyodbc as pyo
import pandas as pd
import os

In [203]:
DRIVER_NAME='SQL SERVER'
USERNAME='sa'
SERVER_NAME = 'DESKTOP-RJ6N7AA\MSSQLSERVER1'
DATABASE_NAME='python_scenarios'
PASSWORD='1234567890'

In [204]:
connectionString = f'DRIVER={{SQL Server}};SERVER={SERVER_NAME};DATABASE={DATABASE_NAME};UID={USERNAME};PWD={PASSWORD}'

In [205]:
#connecting db
con=pyo.connect(connectionString)
print(con)

In [206]:
#create database
cur=con.cursor()
#cur.execute('CREATE DATABASE Deepak')
#cur.execute('CREATE TABLE')

# Extracting customer information from a SQL Server database for reporting purposes.

In [207]:
file_path=r"C:\Users\Deepak\Desktop\python_practice\customer_data_cleaned.csv"
df=pd.read_csv(file_path)

print(df.columns)
df.drop(columns=['country_code'])
df['phone']=df['phone'].str[-10:]
df['registration_date']=pd.to_datetime(df['registration_date'], dayfirst=True)

Index(['customer_id', 'name', 'email', 'phone', 'address', 'registration_date',
       'loyalty_status', 'country_code'],
      dtype='object')


In [208]:
with pyo.connect(connectionString) as conn:
    cursor=conn.cursor()
    create_table_query='''
                        create table customers(
                        customer_id int primary key,
                        customer_name varchar(max),
                        email varchar(50),
                        phone varchar(max),
                        address varchar(max),
                        registration_date date,
                        loyalty_status varchar(10)
                        )
                    '''
    cursor.execute(create_table_query)
   

In [209]:

print(df.dtypes)

customer_id                   int64
name                         object
email                        object
phone                        object
address                      object
registration_date    datetime64[ns]
loyalty_status               object
country_code                 object
dtype: object


In [210]:
with pyo.connect(connectionString) as con:
    cursor=con.cursor()
    insert_values='''
                    insert into customers(customer_id,customer_name,email,phone,address,registration_date,loyalty_status)
                    values(?,?,?,?,?,?,?)
                    '''
    for index,row in df.iterrows():
        cursor.execute(insert_values,
                                    row['customer_id'],
                                    row['name'],
                                    row['email'],
                                    row['phone'],
                                    row['address'],
                                    row['registration_date'],
                                    row['loyalty_status']
                        )

# inserting transactional table


In [211]:
df1=pd.read_csv(r"C:\Users\Deepak\Desktop\python_practice\transaction_table_cleaned.csv")

df1['transaction_date']=pd.to_datetime(df1['transaction_date'])
df1.columns = df1.columns.str.strip() 
df1.columns
print(df1.dtypes)
print(df1.columns.tolist())

df1.columns = df1.columns.str.strip().str.lower()
print(df1.columns.tolist())  # check again


transaction_id               int64
customer_id                  int64
amount                     float64
transaction_date    datetime64[ns]
product_category            object
payment_method              object
store_location              object
dtype: object
['transaction_id', 'customer_id', 'amount', 'transaction_date', 'product_category', 'payment_method', 'store_location']
['transaction_id', 'customer_id', 'amount', 'transaction_date', 'product_category', 'payment_method', 'store_location']


In [212]:
with pyo.connect(connectionString) as conn:
    cursor=conn.cursor()
    create_table_query='''
                        create table transactions(
                        transaction_id int primary key,
                        customer_id int,
                        amount float,
                        transaction_date date,
                        product_category varchar(max),
                        payment_method varchar(max),
                        store_location varchar(max),
                        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
                        )
                    '''
    cursor.execute(create_table_query)

In [213]:
df.columns = df.columns.str.strip().str.lower()  # normalize column names

with pyo.connect(connectionString) as con:
    cursor = con.cursor()
    
    insert_query = '''
        INSERT INTO transactions(transaction_id, customer_id, amount, transaction_date, product_category, payment_method, store_location)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    '''
    
    for _, row in df1.iterrows():
        cursor.execute(insert_query,
                       row['transaction_id'],
                       row['customer_id'],
                       row['amount'],
                       row['transaction_date'],
                       row['product_category'],
                       row['payment_method'],
                       row['store_location'])
    con.commit()


# uploading orders table


In [214]:
df2=pd.read_csv(r"C:\Users\Deepak\Desktop\python_practice\order_data 1.csv")
df2['order_date']=pd.to_datetime(df2['order_date'], dayfirst=True)
df2.dtypes

C:\Users\Deepak\AppData\Local\Temp\ipykernel_5112\1076480244.py:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df2['order_date']=pd.to_datetime(df2['order_date'], dayfirst=True)


order_id                    object
customer_id                  int64
order_date          datetime64[ns]
order_amount               float64
order_status                object
product_category            object
dtype: object

In [215]:
with pyo.connect(connectionString) as conn:
    cursor = conn.cursor()
    create_table_query = '''
        CREATE TABLE orders (
            order_id VARCHAR(50) PRIMARY KEY,
            customer_id INT,
            order_date DATE,
            order_amount FLOAT,
            order_status VARCHAR(50),
            product_category VARCHAR(100)
        )
    '''
    cursor.execute(create_table_query)


In [216]:
with pyo.connect(connectionString) as con:
    cursor = con.cursor()
    
    insert_query = '''
        INSERT INTO orders(order_id, customer_id, order_date, order_amount, order_status, product_category)
        VALUES (?, ?, ?, ?, ?, ?)
    '''
    
    for _, row in df2.iterrows():
        cursor.execute(insert_query,
                       row['order_id'],
                       row['customer_id'],
                       row['order_date'],
                       row['order_amount'],
                       row['order_status'],
                       row['product_category'])
    con.commit()

# Data Transformation Requirements
# naming processing